### **Laboratorio 1**

### **Carga de Datos de Importacion Hidrocarburos**

In [90]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro

### **Datos de Importacion** 

In [91]:
df_import_2024 = pd.read_excel("IMPORTACION-HIDROCARBUROS-VOLUMEN-2024-12.xlsx", skiprows=6)
df_import_2025 = pd.read_excel("IMPORTACION-HIDROCARBUROS-VOLUMEN-2025-05.xlsx", skiprows=6)

print(df_import_2024.columns.tolist())

['Fecha', 'Aceites lubricantes', 'Asfalto ', 'Bunker', 'Ceras', 'Combustible turbo jet', 'Diesel bajo azufre', 'Diesel ultra bajo azufre', 'Gas licuado de petróleo', 'Gasolina de aviación', 'Gasolina regular', 'Gasolina superior', 'Grasas lubricantes', 'Kerosina', 'Mezclas oleosas', 'Naftas', 'Petcoke', 'Petróleo crudo', 'Solventes', 'Butano', 'Diesel alto azufre', 'MTBE', 'Orimulsión', 'Petróleo reconstituido', 'Total importación']


In [92]:
# Unir Alto Azufre y Bajo Azufre para Diésel
df_import_2024["Diesel"] = df_import_2024["Diesel alto azufre"] + df_import_2024["Diesel bajo azufre"]
df_import_2025["Diesel"] = df_import_2024["Diesel alto azufre"] + df_import_2024["Diesel bajo azufre"]


In [93]:
# Unir los dos datasets
df_total = pd.concat([df_import_2024, df_import_2025], ignore_index=True)

# Convertir columna de fecha (ajustar nombre si es diferente)
df_total["Fecha"] = pd.to_datetime(df_total["Fecha"], errors="coerce")

# Seleccionar columnas de interés
cols = ["Gasolina regular", "Gasolina superior", "Diesel", "Gas licuado de petróleo"]
df_comb = df_total[["Fecha"] + cols].dropna()

### **Datos de Precios**

In [94]:

# Leer todas las hojas del archivo
xls = pd.ExcelFile("Precios-Promedio-Nacionales-Diarios-2025-1.xlsx")
df_list = []

for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet, skiprows=7)
    df = df.dropna(how='all')  # Eliminar filas completamente vacías
    df_list.append(df)

# Unir todas las hojas en un solo DataFrame
df_total = pd.concat(df_list, ignore_index=True)
columnas_deseadas = ['FECHA', 'Superior', 'Regular', 'Diesel', 'Glp Cilindro 25Lbs.']
df_Precios = df_total[columnas_deseadas]
# Ver los primeros datos
df_Precios.head()


,FECHA,Superior,Regular,Diesel,Glp Cilindro 25Lbs.
0,2025-01-01 00:00:00,28.95,27.45,25.47,110.0
1,2025-01-02 00:00:00,28.95,27.45,25.47,110.0
2,2025-01-03 00:00:00,28.95,27.45,25.47,110.0
3,2025-01-04 00:00:00,28.95,27.45,25.47,110.0
4,2025-01-05 00:00:00,28.95,27.45,25.47,110.0


### **Datos de Consumo**

#### **Consumo 2025**

In [95]:
df = pd.read_excel("VENTAS-HIDROCARBUROS-2025-05.xlsx", skiprows=6)

# Eliminar filas completamente vacías
df = df.dropna(how='all')

# Convertir la columna de fecha antes de filtrar
df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")

# Filtrar columnas deseadas
df_consumo_2025 = df[[
    "Gasolina regular",
    "Gasolina superior",
    "Diesel bajo azufre",
    "Gas licuado de petróleo",
    "Fecha"
]]

# Renombrar la columna
df_consumo_2025 = df_consumo_2025.rename(columns={"Diesel bajo azufre": "Diesel"})

# Verificar
print(df_consumo_2025.head())


   Gasolina regular  Gasolina superior      Diesel  Gas licuado de petróleo  \
0         828269.64          702319.64  1399798.38                581794.25   
1         781113.92          651035.74  1343721.50                586541.02   
2         895068.46          763939.48  1456533.00                589796.81   
3         878741.74          746081.22  1384488.46                581405.92   
4         893158.40          761100.59  1416602.71                594599.75   

       Fecha  
0 2025-01-01  
1 2025-02-01  
2 2025-03-01  
3 2025-04-01  
4 2025-05-01  


#### **Consumo hasta 2024**

In [96]:


df = pd.read_excel("CONSUMO-HIDROCARBUROS-2024-12.xlsx", skiprows=6)
df = df.dropna(how='all')


df["Diesel"] = df["Diesel bajo azufre"] + df["Diesel alto azufre"]


df["Fecha"] = pd.to_datetime(df["Fecha"], errors="coerce")


df_consumo_hasta2024 = df[[
    "Gasolina regular",
    "Gasolina superior",
    "Diesel",
    "Gas licuado de petróleo",
    "Fecha"
]]


print(df_consumo_hasta2024.head())


   Gasolina regular  Gasolina superior     Diesel  Gas licuado de petróleo  \
0         202645.20          308156.82  634667.06            194410.476190   
1         205530.96          307766.31  642380.66            174710.552381   
2         229499.56          331910.29  699807.25            189234.066667   
3         210680.40          315648.08  586803.98            174330.607143   
4         208164.34          319667.97  656948.20            191745.147619   

       Fecha  
0 2000-01-01  
1 2000-02-01  
2 2000-03-01  
3 2000-04-01  
4 2000-05-01  


### **Unir ambos**

In [97]:
# Unir df_2024 y df_2025 verticalmente
df_consumo_total = pd.concat([df_consumo_hasta2024, df_consumo_2025], ignore_index=True)


In [98]:
df_consumo_total.head()

,Gasolina regular,Gasolina superior,Diesel,Gas licuado de petróleo,Fecha
0,202645.20,308156.82,634667.06,194410.476190,2000-01-01
1,205530.96,307766.31,642380.66,174710.552381,2000-02-01
2,229499.56,331910.29,699807.25,189234.066667,2000-03-01
3,210680.40,315648.08,586803.98,174330.607143,2000-04-01
4,208164.34,319667.97,656948.20,191745.147619,2000-05-01
